In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import numpy as np

In [2]:
query = """

SELECT DISTINCT ?label_country ?label_indicator ?value ?year WHERE {

    ?country a <http://worldbank.org/Country> .
    ?country <http://www.w3.org/2002/07/owl#sameAs> ?sameAsCountry .
    ?country <http://worldbank.org/hasAnnualIndicatorEntry> ?annualIndicator .
    ?annualIndicator <http://www.w3.org/2002/07/owl#hasValue> ?value .
    ?annualIndicator <http://www.w3.org/2006/time#year> ?year .
    ?annualIndicator <http://worldbank.org/hasIndicator> ?indicator .
    ?indicator <http://www.w3.org/2000/01/rdf-schema#label> ?label_indicator .
    ?country <http://www.w3.org/2000/01/rdf-schema#label> ?label_country .
    
    FILTER(?indicator in (<http://worldbank.org/Indicator/NY.GDP.MKTP.CD>,
    <http://worldbank.org/Indicator/NE.DAB.TOTL.CD>, <http://worldbank.org/Indicator/NY.GDP.PCAP.CN>,
    <http://worldbank.org/Indicator/SI.POV.GINI>, <http://worldbank.org/Indicator/NV.IND.TOTL.ZS>,
    <http://worldbank.org/Indicator/EN.ATM.CO2E.PC>, <http://worldbank.org/Indicator/SH.STA.BASS.ZS>,
    <http://worldbank.org/Indicator/SH.STA.DIAB.ZS>, <http://worldbank.org/Indicator/SH.TBS.INCD>,
    <http://worldbank.org/Indicator/SP.DYN.LE00.IN>, <http://worldbank.org/Indicator/SP.POP.GROW>,
    <http://worldbank.org/Indicator/SH.HIV.INCD.TL.P3>, <http://worldbank.org/Indicator/SH.MED.BEDS.ZS>,
    <http://worldbank.org/Indicator/SH.PRG.ANEM>, <http://worldbank.org/Indicator/SH.PRV.SMOK>,
    <http://worldbank.org/Indicator/SH.STA.BASS.ZS>)
    AND (?year=2019))

}
"""

In [3]:
def convert_to_category(world_bank, indicator, lower):
    labels = ['low', 'mediumLow', 'medium', 'mediumHigh', 'high']
    if lower:
        labels.reverse()
    df = world_bank.loc[world_bank.indicator==indicator]
    df["value"] = pd.to_numeric(df["value"], downcast="float")
    df = df.sort_values(by='value')  # , ascending=boolean
    a = list(df['value'].values)
    n_split = np.array_split(a, 5)
    category = pd.cut(df.value, bins=[min(n_split[0])-1, max(n_split[0]), max(n_split[1]), max(n_split[2]),
                                      max(n_split[3]), max(n_split[4])], labels=labels)
    df.insert(3, 'category_indicator',category)
    df.drop(columns='value', inplace=True)
    return df

In [4]:
sparql = SPARQLWrapper("https://labs.tib.eu/sdm/worldbank_endpoint/sparql")
world_bank = []

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


for r in results['results']['bindings']:
    row = {'country': r['label_country']['value'].replace(',', ''),
           'indicator': r['label_indicator']['value'].replace(',', ''),
           'value': r['value']['value']}
    
    world_bank.append(row)

world_bank = pd.DataFrame.from_dict(world_bank)
world_bank = world_bank.drop_duplicates()
display(world_bank.shape, world_bank.head(2))

(2328, 3)

,country,indicator,value
0,Malawi,GDP (current US$),1.10254e+10
2,Malawi,Industry (including construction) value added ...,18.5355


In [5]:
world_bank.indicator.unique().tolist()

['GDP (current US$)',
 'Industry (including construction) value added (% of GDP)',
 'GDP per capita (current LCU)',
 'Gross national expenditure (current US$)',
 'Prevalence of current tobacco use (% of adults)',
 'Incidence of tuberculosis (per 100000 people)',
 'Prevalence of anemia among pregnant women (%)',
 'Life expectancy at birth total (years)',
 'People using at least basic sanitation services (% of population)',
 'Population growth (annual %)',
 'Incidence of HIV all (per 1000 uninfected population)',
 'Gini index',
 'CO2 emissions (metric tons per capita)',
 'Hospital beds (per 1000 people)']

In [6]:
len(world_bank.country.unique())

215

In [7]:
country_ind = pd.DataFrame()
for indicator in world_bank.indicator.unique().tolist():
    lower = False
    if indicator in ['CO2 emissions (metric tons per capita)', 'Incidence of tuberculosis (per 100000 people)',
                    'Prevalence of anemia among pregnant women (%)',
                     'Incidence of HIV all (per 1000 uninfected population)',
                     'Prevalence of current tobacco use (% of adults)']:
        lower = True
    country_ind = pd.concat([country_ind, convert_to_category(world_bank, indicator, lower)])

country_ind

/tmp/ipykernel_19821/4058174006.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["value"] = pd.to_numeric(df["value"], downcast="float")


,country,indicator,category_indicator
436,Tuvalu,GDP (current US$),low
86,Nauru,GDP (current US$),low
1684,Kiribati,GDP (current US$),low
1846,Marshall Islands,GDP (current US$),low
141,Palau,GDP (current US$),low
...,...,...,...
4330,New Zealand,Hospital beds (per 1000 people),mediumLow
3331,Denmark,Hospital beds (per 1000 people),medium
3734,Iceland,Hospital beds (per 1000 people),medium
3989,Luxembourg,Hospital beds (per 1000 people),mediumHigh


In [8]:
country_ind = country_ind.drop_duplicates()
country_ind.shape

(2328, 3)

In [9]:
country_ind.to_csv('dataset/country_ind.csv', index=None)